# Example with big amount of scrapping

# FLIPKART_SCRAPER.py

In [1]:
import requests  # required for HTTP requests: pip install requests
from bs4 import BeautifulSoup  # required for HTML and XML parsing
# from datetime import date
import time


class FlipkartScrapper:
    headers = {
        "User-Agent": 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.149 Safari/537.36'
    }
    search_item = ''

    def __init__(self, search_item):
        self.search_item = search_item

    # find desired class for link of item
    def findClass(self):
        r = requests.get('https://www.flipkart.com/search?q=' + self.search_item + '&page=1', headers=self.headers)
        soup = BeautifulSoup(r.content, 'html.parser')
        class1 = '_2cLu-l'
        class2 = '_2mylT6'
        class3 = '_31qSD5'
        if bool(soup.find('a', attrs={'class': class1})):
            return class1
        else:
            if bool(soup.find('a', attrs={'class': class2})):
                return class2
            else:
                return class3

    # scrape all the links from website
    def flipkartLinkScrapper(self, page_start, page_end):
        temp_class = self.findClass()
        time.sleep(1)   # sleep to so website can't detact it
        links = []
        for i in range(page_start, page_end):
            # https://www.flipkart.com/search?q=Redmi&page=5
            r = requests.get('https://www.flipkart.com/search?q=' + self.search_item + '&page=' + str(i), headers=self.headers)
            soup = BeautifulSoup(r.content, 'html.parser')
            temp = soup.findAll('a', attrs={'class': temp_class})
            links += temp
            print(str(i + 1) + ' no. page has ' + str(len(temp)) + ' links')
            time.sleep(0.1)

        links_set = set(links)
        return links_set

    def scrapeFlipkartProductInfo(self, link):
        data = []
        r = requests.get(link, headers=self.headers)  # ,proxies=proxies)
        soup = BeautifulSoup(r.content, 'html.parser')
        time.sleep(0.01)
        title = soup.find('span', attrs={'class': '_35KyD6'})
        rating = soup.find('div', attrs={'class': 'hGSR34'})
        rating_num = soup.find('span', attrs={'class': '_38sUEc'})
        original_price = soup.find('div', attrs={'class': '_3auQ3N _1POkHg'})
        discounted_price = soup.find('div', attrs={'class': '_1vC4OE _3qQ9m1'})

        if title is not None:
            data.append(title.text.strip().replace('\xa0', ' '))
        else:
            data.append('unknown')
        if original_price is not None:
            data.append(original_price.text.strip()[1:])
        else:
            data.append('-1')
        if discounted_price is not None:
            data.append(discounted_price.text.strip()[1:])
        else:
            data.append('-1')
        if rating is not None:
            data.append(rating.text.strip())
        else:
            data.append('-1')
        if rating_num is not None:
            r_n = rating_num.text.replace('\xa0', ' ')
            data.append(r_n[: r_n.find('Ratings')].strip())
        else:
            data.append('-1')

        data.append(link)
        return data


# WEB_CRAWLER.py

In [2]:
from FLIPKART_SCRAPER import FlipkartScrapper
from threading import Thread
from queue import *
import time
import numpy as np


class WebCrawler:
    start_time = 0
    URL = ''
    objScrapper = ''
    objSaver = ''
    links = []
    list_of_links = []
    links_queue = Queue()
    thread_list = []
    thread_list_product = []
    data = []
    no_of_threads = 1

    def __init__(self, search_item, website, start_pages=0, end_pages=1, no_of_threads=1):
        self.search_item = search_item
        self.start_pages = start_pages
        self.end_pages = end_pages
        self.no_of_threads = no_of_threads
        self.website = website
        if self.website.upper() == 'FLIPKART':
            self.URL = 'https://www.flipkart.com'
            self.objScrapper = FlipkartScrapper(self.search_item)

    def startCounting(self):
        self.start_time = time.time()

    def endCounting(self):
        temp = self.start_time
        self.start_time = 0
        return int(time.time() - temp)

    # function use with multi-threading to returned save data
    def scrapeLinks(self, start, end):
        if self.website.upper() == 'FLIPKART':
            result = list(self.objScrapper.flipkartLinkScrapper(start, end))
        for r in result:
            self.links_queue.put(self.URL + r['href'])

    def getAllItemLinks(self):
        for i in np.arange(self.start_pages, self.end_pages, self.no_of_threads):
            # if last k pages are less than our counter
            if i + self.no_of_threads > self.end_pages:
                j = self.end_pages
            else:
                j = i + self.no_of_threads
            t = Thread(target=self.scrapeLinks, args=(i, j))
            t.start()
            self.thread_list.append(t)

        for th in self.thread_list:
            th.join()

        while not self.links_queue.empty():
            self.links.append(self.links_queue.get())
        print(len(self.links))

    def scrapMultipleItemInfo(self, idx):
        # idxx = idx[0]
        for datax in self.list_of_links[idx]:
            print(self.links.index(datax))
            temp_data = self.objScrapper.scrapeFlipkartProductInfo(datax)
            self.data.append(temp_data)

    def Cloning(self, li1):
        li_copy = []
        li_copy.extend(li1)
        return li_copy

    def sliceData(self):
        list_copy = self.Cloning(self.links)
        n = int(len(list_copy) / self.no_of_threads)
        print('n' + str(n))
        while True:
            temp_list = list_copy[0:n]
            self.list_of_links.append(temp_list)
            list_copy = list_copy[n:]
            if n > len(list_copy):
                temp_list += list_copy
                break

    def getAllItemInfo(self):
        self.sliceData()
        no_of_list = len(self.list_of_links)
        print(len(self.list_of_links))
        for idx in range(no_of_list):
            t = Thread(target=self.scrapMultipleItemInfo, args=([idx]))
            t.start()
            self.thread_list_product.append(t)

        for th in self.thread_list_product:
            th.join()

    def scrapIt(self):
        self.startCounting()
        self.getAllItemLinks()
        self.getAllItemInfo()
        x = self.endCounting()
        print('Total Excution time : ', end='\t')
        print(x, end='\t')
        print('seconds')


# SAVE_DATA.py

In [3]:
import csv


class SaveData:

    def __init__(self, data=None, name_of_file='tempCSV'):
        if data is None:
            data = [[0, 0, 0, 0, 0, 0]]
        self.data = data
        self.name_of_file = name_of_file.upper() + '.csv'

    def saveDataInCSVFile(self):
        with open(self.name_of_file, 'w', newline='') as file:
            writer = csv.writer(file)
            writer.writerow(['name', 'original_price', 'discounted_price', 'rating', 'rating_number', 'link'])
            for d in self.data:
                if int(d[1].replace(',', '')) == -1:
                    d[1] = d[2]
                if int(d[2].replace(',', '')) == -1:
                    d[2] = d[1]
                writer.writerow([d[0], d[1], d[2], d[3], d[4], d[5]])

    def appendDataInCSVFile(self):
        with open(self.name_of_file, 'a', newline='') as file:
            writer = csv.writer(file)
            for d in self.data:
                if int(d[1].replace(',', '')) == -1:
                    d[1] = d[2]
                if int(d[2].replace(',', '')) == -1:
                    d[2] = d[1]
                writer.writerow([d[0], d[1], d[2], d[3], d[4], d[5]])


In [4]:
# name of variable explains it
item = 'smartphone'
website = 'flipkart'
start_pages = 0
end_pages = 24
no_of_threads = 4
obj = WebCrawler(item, website, start_pages, end_pages, no_of_threads)

In [5]:
obj.scrapIt() # this Example we're printing product number which are scraped.

21 no. page has 24 links
5 no. page has 24 links
13 no. page has 24 links
9 no. page has 24 links
1 no. page has 24 links
17 no. page has 24 links
22 no. page has 24 links
14 no. page has 24 links
6 no. page has 24 links
10 no. page has 24 links
2 no. page has 24 links
18 no. page has 24 links
23 no. page has 24 links
15 no. page has 24 links
7 no. page has 24 links
3 no. page has 24 links
11 no. page has 24 links
19 no. page has 24 links
24 no. page has 24 links
8 no. page has 24 links
16 no. page has 24 links
12 no. page has 24 links
20 no. page has 24 links
4 no. page has 24 links
576
n144
4
0
144
288
432
289
433
1
145
434146
290

2
435
147
291
3
436
4
148
292
437
5
149
293
438
6
150
294
439
7
151
295
440
8
152
296
9
153
297
441
10
154
298
442
11
155
299
443
12
156
300
444
13
157
445
14
301
158
15
302
446
159
16
447
303
160
17
448
304
18
449
161
305
19
306
450
162
20
307
163
21
451
308
164
22
452
309
165
23
453
310
24
166
25
454
167
311
26
168
455
312
27
456
169
313
28
170
457
314
2

In [6]:
saveObj = SaveData(obj.data,'Example') # sve data in EXAMPLE.CSV file

In [7]:
saveObj.saveDataInCSVFile()

# FIND_BEST_PRODUCT.py

In [9]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt


# Set some default plot styles to make things look nice
# matplotlib.rcParams['figure.figsize'] = (20.0, 10.0)
# plt.style.use('bmh')
# pd.set_option('display.max_rows', None)
# pd.set_option('display.max_columns', None)
# pd.set_option('display.width', None)
# pd.set_option('display.max_colwidth', -1)


class FindBestItem:
    ans = ''

    def __init__(self, name_of_file, min_price, max_price):
        self.name_of_file = name_of_file
        self.min_price = min_price
        self.max_price = max_price
        self.data = pd.read_csv(self.name_of_file)
        self.filterData()
        self.typeConversion()
        self.dropDuplicates()

    # removing unwanted part of data/ polishing data
    def filterData(self):
        self.data['original_price'] = self.data['original_price'].str.replace(r',', '')
        self.data['discounted_price'] = self.data['discounted_price'].str.replace(r',', '')
        self.data['rating_number'] = self.data['rating_number'].str.replace(r',', '')

    # TYPE CONVERSION
    def typeConversion(self):
        self.data['original_price'] = pd.to_numeric(self.data['original_price'])
        self.data['discounted_price'] = pd.to_numeric(self.data['discounted_price'])
        self.data['rating'] = pd.to_numeric(self.data['rating'])
        self.data['rating_number'] = pd.to_numeric(self.data['rating_number'])

    def dropDuplicates(self):
        self.data = self.data.drop_duplicates(subset='name')

    # name itself tells it
    def filter_products_based_on_discounted_price(self):
        minimum_rating = self.data['rating'].mean()
        min_buyer_number = self.data['rating_number'].mean()
        temp = self.data[
            ((self.data['discounted_price'] > self.min_price) & (self.data['discounted_price'] <= self.max_price))]
        temp = temp[((temp['rating'] > minimum_rating) & (temp['rating_number'] > min_buyer_number))].sort_values(
            'rating', ascending=False).head(10)
        self.ans = temp

    def writeAnswer(self):
        self.ans.to_csv('ans_of_' + self.name_of_file, index=False)

    def findBest(self):
        self.filter_products_based_on_discounted_price()
        self.writeAnswer()

In [11]:
Obj1 = FindBestItem('EXAMPLE.csv', 10000, 30000) # name of file to get data, min_price, max_price

In [12]:
#here we deomostrate each part of funcation 'filter_products_based_on_discounted_price'

In [13]:
Obj1.data

,name,original_price,discounted_price,rating,rating_number,link
0,"Meizu C9 (Gold, 16 GB) (2 GB RAM)",6999,3790,4.1,1261,https://www.flipkart.com/meizu-c9-gold-16-gb/p...
1,"Honor 10 Lite (Sapphire Blue, 32 GB) (3 GB RAM)",13999,8499,4.4,26479,https://www.flipkart.com/honor-10-lite-sapphir...
2,"Nokia 7.2 (Cyan Green, 64 GB) (4 GB RAM)",21099,16399,4.2,1090,https://www.flipkart.com/nokia-7-2-cyan-green-...
3,"Realme X2 Pro (Neptune Blue, 64 GB) (6 GB RAM)",29999,29999,4.5,441,https://www.flipkart.com/realme-x2-pro-neptune...
4,"POCO F1 (Graphite Black, 128 GB) (6 GB RAM)",24999,18999,4.5,236950,https://www.flipkart.com/poco-f1-graphite-blac...
...,...,...,...,...,...,...
571,"Vivo V11 (Nebula Purple, 64 GB) (6 GB RAM)",24990,24990,4.5,17585,https://www.flipkart.com/vivo-v11-nebula-purpl...
572,"Realme 5 Pro (Sparkling Blue, 64 GB) (6 GB RAM)",15999,14999,4.5,72665,https://www.flipkart.com/realme-5-pro-sparklin...
573,"Coolpad Note 3S (Gold, 32 GB) (3 GB RAM)",9999,9999,3.8,1003,https://www.flipkart.com/coolpad-note-3s-gold-...
574,"Redmi Note 8 (Moonlight White, 128 GB) (6 GB ...",15999,13390,4.4,7605,https://www.flipkart.com/redmi-note-8-moonligh...


In [14]:
minimum_rating = Obj1.data['rating'].mean()
min_buyer_number = Obj1.data['rating_number'].mean()
print(minimum_rating,min_buyer_number)

4.2626134301270415 53753.08166969147


In [15]:
temp = Obj1.data[((Obj1.data['discounted_price'] > Obj1.min_price) & (Obj1.data['discounted_price'] <= Obj1.max_price))]
temp

,name,original_price,discounted_price,rating,rating_number,link
2,"Nokia 7.2 (Cyan Green, 64 GB) (4 GB RAM)",21099,16399,4.2,1090,https://www.flipkart.com/nokia-7-2-cyan-green-...
3,"Realme X2 Pro (Neptune Blue, 64 GB) (6 GB RAM)",29999,29999,4.5,441,https://www.flipkart.com/realme-x2-pro-neptune...
4,"POCO F1 (Graphite Black, 128 GB) (6 GB RAM)",24999,18999,4.5,236950,https://www.flipkart.com/poco-f1-graphite-blac...
5,"OPPO A31 (Fantasy White, 64 GB) (4 GB RAM)",12990,12490,4.4,196,https://www.flipkart.com/oppo-a31-fantasy-whit...
9,"Samsung Galaxy M30s (Blue, 128 GB) (6 GB RAM)",18500,17899,4.3,340,https://www.flipkart.com/samsung-galaxy-m30s-b...
...,...,...,...,...,...,...
570,"Realme 2 Pro (Black Sea, 64 GB) (4 GB RAM)",13990,11490,4.5,382562,https://www.flipkart.com/realme-2-pro-black-se...
571,"Vivo V11 (Nebula Purple, 64 GB) (6 GB RAM)",24990,24990,4.5,17585,https://www.flipkart.com/vivo-v11-nebula-purpl...
572,"Realme 5 Pro (Sparkling Blue, 64 GB) (6 GB RAM)",15999,14999,4.5,72665,https://www.flipkart.com/realme-5-pro-sparklin...
574,"Redmi Note 8 (Moonlight White, 128 GB) (6 GB ...",15999,13390,4.4,7605,https://www.flipkart.com/redmi-note-8-moonligh...


In [16]:
temp = temp[((temp['rating'] > minimum_rating) & (temp['rating_number'] > min_buyer_number))].sort_values('rating', ascending=False)
temp  

,name,original_price,discounted_price,rating,rating_number,link
4,"POCO F1 (Graphite Black, 128 GB) (6 GB RAM)",24999,18999,4.5,236950,https://www.flipkart.com/poco-f1-graphite-blac...
275,"Vivo Z1Pro (Sonic Blue, 64 GB) (6 GB RAM)",17990,14990,4.5,126065,https://www.flipkart.com/vivo-z1pro-sonic-blue...
296,"Realme XT (Pearl Blue, 128 GB) (8 GB RAM)",19999,19999,4.5,70084,https://www.flipkart.com/realme-xt-pearl-blue-...
304,"Redmi Note 7 Pro (Space Black, 64 GB) (4 GB RAM)",15999,10999,4.5,783264,https://www.flipkart.com/redmi-note-7-pro-spac...
322,"Redmi Note 7 Pro (Nebula Red, 64 GB) (6 GB RAM)",16999,12999,4.5,254178,https://www.flipkart.com/redmi-note-7-pro-nebu...
...,...,...,...,...,...,...
292,"Realme 5s (Crystal Red, 128 GB) (4 GB RAM)",12999,11999,4.4,222936,https://www.flipkart.com/realme-5s-crystal-red...
256,"Realme 5s (Crystal Purple, 64 GB) (4 GB RAM)",11999,11999,4.4,222936,https://www.flipkart.com/realme-5s-crystal-pur...
320,"Redmi Note 4 (Gold, 64 GB) (4 GB RAM)",12999,12999,4.4,1323368,https://www.flipkart.com/redmi-note-4-gold-64-...
326,"Asus ZenFone Max Pro M2 (Blue, 64 GB) (4 GB RAM)",17999,11999,4.3,74779,https://www.flipkart.com/asus-zenfone-max-pro-...


In [18]:
answer = temp.head(10)
answer # output to be display to user

,name,original_price,discounted_price,rating,rating_number,link
4,"POCO F1 (Graphite Black, 128 GB) (6 GB RAM)",24999,18999,4.5,236950,https://www.flipkart.com/poco-f1-graphite-blac...
275,"Vivo Z1Pro (Sonic Blue, 64 GB) (6 GB RAM)",17990,14990,4.5,126065,https://www.flipkart.com/vivo-z1pro-sonic-blue...
296,"Realme XT (Pearl Blue, 128 GB) (8 GB RAM)",19999,19999,4.5,70084,https://www.flipkart.com/realme-xt-pearl-blue-...
304,"Redmi Note 7 Pro (Space Black, 64 GB) (4 GB RAM)",15999,10999,4.5,783264,https://www.flipkart.com/redmi-note-7-pro-spac...
322,"Redmi Note 7 Pro (Nebula Red, 64 GB) (6 GB RAM)",16999,12999,4.5,254178,https://www.flipkart.com/redmi-note-7-pro-nebu...
337,"Realme 5 Pro (Sparkling Blue, 128 GB) (8 GB RAM)",17999,16999,4.5,58622,https://www.flipkart.com/realme-5-pro-sparklin...
340,"Realme 3 Pro (Nitro Blue, 128 GB) (6 GB RAM)",19999,16999,4.5,122154,https://www.flipkart.com/realme-3-pro-nitro-bl...
380,"Redmi Note 7 (Onyx Black, 64 GB) (4 GB RAM)",10799,10799,4.5,173630,https://www.flipkart.com/redmi-note-7-onyx-bla...
383,"Vivo Z1Pro (Sonic Blue, 64 GB) (4 GB RAM)",15990,13990,4.5,313136,https://www.flipkart.com/vivo-z1pro-sonic-blue...
400,"Realme 2 Pro (Black Sea, 64 GB) (6 GB RAM)",15990,13990,4.5,90918,https://www.flipkart.com/realme-2-pro-black-se...


In [20]:
Obj1.findBest() #this fun will find anwser ans save data in file ans_of_EXAMPLE.csv and will be read by GUI.